In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [2]:
## Load dataset
data_path = "../../../Rings_P3"
OB = torch.from_numpy(np.load(data_path + '/ob.npy')).float()
# STATE = torch.from_numpy(np.load(data_path + '/state.npy')).float()
# MU = torch.from_numpy(np.load(data_path + '/mu.npy')).float()
# ANGLE = torch.from_numpy(np.load(data_path + '/angle.npy')).float()

NUM_DATASETS, N, D = OB.shape
K = 4
MCMC_SIZE = 10
SAMPLE_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 300
NUM_HIDDEN_LOCAL = 64
NUM_HIDDEN = 32
NUM_NSS = 8
LEARNING_RATE = 1e-4
CUDA = torch.cuda.is_available()
DEVICE = 0

RECON_SIGMA = torch.ones(1) * 0.3
RADI = torch.ones(1) * 2.0
lg2pi = torch.log(torch.ones(1) * 2 * math.pi)
PATH = 'ag-4rings-p3'

In [3]:
from global_oneshot_mu import *
from local_oneshot_state import *
from local_enc_angle import *
from global_enc_mu import *
from decoder_semi import *
# initialization
oneshot_mu = Oneshot_mu(K, D, NUM_HIDDEN, NUM_NSS, CUDA, DEVICE)
oneshot_state = Oneshot_state(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_angle = Enc_angle(D, NUM_HIDDEN, CUDA, DEVICE)
enc_mu = Enc_mu(K, D, NUM_HIDDEN, NUM_NSS, CUDA, DEVICE)
dec_x = Dec_x(D, NUM_HIDDEN, RECON_SIGMA, CUDA, DEVICE)
if CUDA:
    with torch.cuda.device(DEVICE):
        oneshot_mu.cuda()
        oneshot_state.cuda()
        enc_angle.cuda()
        enc_mu.cuda()
        dec_x.cuda()
optimizer =  torch.optim.Adam(list(dec_x.parameters())+list(oneshot_mu.parameters())+list(oneshot_state.parameters())+list(enc_mu.parameters())+list(enc_angle.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))

models = (oneshot_mu, oneshot_state, enc_angle, enc_mu, dec_x)

In [4]:
oneshot_mu.load_state_dict(torch.load('../results/oneshot-mu-' + PATH))
oneshot_state.load_state_dict(torch.load('../results/oneshot-state-' + PATH))
enc_angle.load_state_dict(torch.load('../results/enc-angle-' + PATH))
enc_mu.load_state_dict(torch.load('../results/enc-mu-' + PATH))
dec_x.load_state_dict(torch.load('../results/dec-x-' + PATH))

In [ ]:
train(models, optimizer, OB, NUM_EPOCHS, MCMC_SIZE, SAMPLE_SIZE, BATCH_SIZE, K, CUDA, DEVICE, PATH)

epoch=0, eubo=-36054.4492, elbo=-42181.1172, ess=4.0018 (319s)
epoch=1, eubo=-35744.2656, elbo=-41808.0469, ess=4.0359 (323s)
epoch=2, eubo=-35607.2930, elbo=-41618.5820, ess=4.0523 (323s)
epoch=3, eubo=-35560.7070, elbo=-41560.2578, ess=4.0749 (323s)
epoch=4, eubo=-35472.8242, elbo=-41417.2617, ess=4.0827 (323s)
epoch=5, eubo=-35389.5273, elbo=-41270.4805, ess=4.0995 (322s)
epoch=6, eubo=-35355.0469, elbo=-41182.5391, ess=4.1073 (324s)
epoch=7, eubo=-35316.0234, elbo=-41094.5664, ess=4.1237 (328s)
epoch=8, eubo=-35321.5273, elbo=-41024.3477, ess=4.1333 (325s)
epoch=9, eubo=-35302.6289, elbo=-40927.3320, ess=4.1420 (325s)
epoch=10, eubo=-35234.9414, elbo=-40839.0664, ess=4.1559 (326s)
epoch=11, eubo=-35257.7812, elbo=-40802.0547, ess=4.1672 (326s)
epoch=12, eubo=-35198.0664, elbo=-40714.9531, ess=4.1820 (326s)
epoch=13, eubo=-35226.9805, elbo=-40685.2500, ess=4.1877 (325s)
epoch=14, eubo=-35182.4258, elbo=-40590.6172, ess=4.1946 (325s)
epoch=15, eubo=-35148.3242, elbo=-40558.4805, ess=

In [ ]:
BATCH_SIZE = 5
MCMC_STEPS = 20 ## 12 is maximum mcmc steps
VIS_INTERVAL = 4
FS = 25
TITLE_FS = 20
OB_MS = 1.0
MU_MARKER = 'X'
MU_MS = 100
OPACITY = 0.8
BOUND = 10
COLORS = ['#EE7733', 'm', '#0077BB', '#009988']
##
DATA_PTR = 5

plot_obj = Plot_MC(BATCH_SIZE, MCMC_STEPS, VIS_INTERVAL, FS, TITLE_FS, OB_MS, MU_MARKER, MU_MS, OPACITY, BOUND, COLORS, CUDA, DEVICE)
plot_obj.plot_results(models, OB, K, DATA_PTR)
plt.savefig('samples.png')

In [ ]:
torch.save(enc_angle.state_dict(), '../results/enc-angle-' + PATH)
torch.save(oneshot_mu.state_dict(), '../results/oneshot-mu-' + PATH)
torch.save(oneshot_state.state_dict(), '../results/oneshot-state-' + PATH)
torch.save(enc_mu.state_dict(), '../results/enc-mu-' + PATH)
torch.save(dec_x.state_dict(), '../results/dec-x-' + PATH)